In [ ]:
import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from time import sleep

# Function to scrape real-time price data for the top 20 cryptocurrencies from Mudrex
def scrape_mudrex():
    url = "https://mudrex.com/all-cryptocurrencies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # table containing cryptocurrency data
    table = soup.find('table', class_='min-w-[25rem] max-w-full md:w-full mb-4 text-[#212529]')

    # Extract top 20 cryptocurrency names and prices
    crypto_data = {}
    for row in table.find_all('tr')[1:21]:
        cols = row.find_all('td')
        name = cols[1].text.strip()
        price = cols[2].text.strip()
        crypto_data[name] = price

    return crypto_data

# Function to update Google Sheet with the scraped data
def update_google_sheet(data):
    # Authenticating with Google Sheets API
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Grull works assignment/grull-works-assignment-0be3d1be47d0.json', scope)
    client = gspread.authorize(creds)

    # Opening the Google Sheet
    sheet = client.open('Crypto Prices').sheet1

    # Clearing the existing data in the sheet
    sheet.clear()

    # Updating the header row
    header = ['Cryptocurrency', 'Price']
    sheet.append_row(header)

    # Updating the sheet with the new data
    for crypto, price in data.items():
        sheet.append_row([crypto, price])

# Main function
def main():
    while True:
        # Scraping data from Mudrex
        crypto_data = scrape_mudrex()

        # Updating the Google Sheet
        update_google_sheet(crypto_data)

        # Waiting for one minute before scraping and updating again
        sleep(60)

if __name__ == "__main__":
    main()
